In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets


In [ ]:
import gymnasium as gym
import math

In [3]:
models_dict = {
    "StableLM Zephyr 3B": "stabilityai/stable-zephyr-3b-dpo"
}

model_id = models_dict["StableLM Zephyr 3B"]

In [4]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
    pipeline,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)#, device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import torch
from transformers import pipeline

# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, device_map="auto", tokenizer=tokenizer, torch_dtype=torch.bfloat16)

In [ ]:
job_description = """You are a large language model playing a game called CartPole. 
You can only answer "right" or "left".
"""

job_description = """You are a large language model playing a game called CartPole. 
        if pole tilts to the right, you should take an action to push the cart right.
        if pole tilts to the left, you should take an action to push the cart left.
        For each prompt you can answer one word: right or left.
        """

In [ ]:
exit_terms = ["stop", "exit"]

def generate(question, max_new_tokens=512):
    messages = [
        {
            "role": "system",
            "content": job_description,
        },
    ]
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    outputs = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    response_start = output.rfind('<|assistant|>')
    text_output = output[response_start + len('<|assistant|>'):]
    print(text_output)
    return text_output

In [9]:
generate("Hi!")


Hello! How can I help you today. If you have any questions about language, learning, or any topic, feel free to ask.

For example, if you want to learn a new language, I can help you find resources, suggest exercises, and provide vocabulary words. If you need help with a particular topic like math, science, or history, I can explain concepts and provide information. If you want to know more about a specific book or movie, I can give you a summary and share my thoughts.

Just let me know what you need help with, and I'll do my best to assist you.


In [58]:
env = gym.make('CartPole-v1', render_mode="human")

for i_episode in range(20):
    observation = env.reset()
    rad = observation[0][2]
    rewards = 0
    for t in range(250):
        env.render()

        #action = env.action_space.sample()
        
        degrees = round(math.degrees(rad), 2)
        direction = "right"
        if degrees < 0:
            direction = "left"
            degrees = degrees * -1

        prompt = """You are a large language model playing a game called CartPole. 
        When the pole tilts to the right in the CartPole game, you should take an action to push the cart to the right to counterbalance the tilt and maintain stability.
        When the pole tilts to the left in the CartPole game, you should take an action to push the cart to the left to counterbalance the tilt and maintain stability.
        For each promt you can answer only: "right" or "left".
        Instruct: In which direction the cart of a cart pole should be moved if the pole direction is """ + str(degrees) + """ degrees off to the """ + direction + """. 
        """
        
        prompt = """You are a large language model playing a game called CartPole. 
        if pole tilts to the right, you should take an action to push the cart to the right.
        if pole tilts to the left, you should take an action to push the cart to the left.
        For each prompt you can answer only: "right" or "left".
        Instruct: In which direction the cart of a cart pole should be moved if the pole direction is """ + str(degrees) + """ degrees off to the """ + direction + """. 
        """
        
        prompt = """You are a large language model playing a game called CartPole. 
        if pole tilts to the right, you should take an action to push the cart to the right.
        if pole tilts to the left, you should take an action to push the cart to the left.
        For each prompt you can answer only: "right" or "left".
        Instruct: The pole tilts """ + direction + """, should we push the cart left or right? anwer one word: 'left' or 'right'. 
        """
        prompt = """
        The pole tilts """ + direction + """, should we push the cart left or right? anwer one word: left or right. 
        """
        
        prompt = """
        The pole tilts """ + direction + """. 
        Should we push the cart left or right? anwer in one word. 
        """

        output = generate(prompt, 5)
        
        action = 0
        if "right" in output.lower():
            action = 1
        print(str(degrees) + " " + direction + " action: " + str(action))
        observation, reward, terminated, truncated, info = env.step(action)
        rad = observation[2]
        rewards += reward

        if terminated:
            print('Episode finished after {} timesteps, total rewards {}'.format(t+1, rewards))
            break

env.close()


left
0.07 left action: 0

left
0.1 left action: 0

right
0.21 right action: 1

right
0.84 right action: 1

right
1.15 right action: 1

right
1.12 right action: 1

right
0.77 right action: 1

right
0.09 right action: 1

left
0.93 left action: 0

left
2.27 left action: 0

left
3.29 left action: 0

left
3.99 left action: 0

left
4.37 left action: 0

left
4.44 left action: 0

left
4.21 left action: 0

left
3.67 left action: 0

left
2.82 left action: 0

left
1.67 left action: 0

left
0.19 left action: 0

right
1.61 right action: 1

right
3.75 right action: 1

right
5.56 right action: 1

right
7.05 right action: 1

right
8.25 right action: 1

right
9.17 right action: 1

right
9.8 right action: 1

right
10.16 right action: 1

right
10.25 right action: 1

right
10.07 right action: 1

right
9.63 right action: 1

right
8.92 right action: 1

right
7.94 right action: 1

right
6.69 right action: 1

right
5.16 right action: 1

right
3.33 right action: 1

right
1.21 right action: 1

left
1.24 left a

KeyboardInterrupt: 

In [57]:
env.close()